In [ ]:
import ee

# Initialize the Earth Engine API.
ee.Authenticate(force = True)
ee.Initialize(project='ee-hadat')

In [1]:
import os
import rasterio
from rasterio.warp import transform_bounds

def get_region_coordinates(tif_file_path):
    """
    Opens a GeoTIFF file and returns the coordinates of its bounds as a list of
    [x, y] pairs in the EPSG:3857 system. Coordinates are ordered as:
    [top-right, bottom-right, bottom-left, top-left, top-right] (closing the polygon).
    """
    with rasterio.open(tif_file_path) as dataset:
        bounds = dataset.bounds  # (left, bottom, right, top)
        # Transform bounds if dataset's CRS is not EPSG:3857.
        if dataset.crs.to_string() != 'EPSG:3857':
            bounds = transform_bounds(dataset.crs, 'EPSG:3857', *bounds)
        
        # Create coordinates list in the specified order.
        coordinates = [
            [bounds[2], bounds[1]],  # Top-right (right, bottom)
            [bounds[2], bounds[3]],  # Bottom-right (right, top)
            [bounds[0], bounds[3]],  # Bottom-left (left, top)
            [bounds[0], bounds[1]],  # Top-left (left, bottom)
            [bounds[2], bounds[1]]   # Closing the loop (same as first point)
        ]
        return coordinates

def read_region_coordinates(folder_path):
    """
    Reads all TIFF images in the given folder (ignoring non-TIF files), extracts the region name
    from the file name, gets the image bounds, and returns a dictionary with keys as region names
    and values as the coordinates (in EPSG:3857) of that image.
    
    Assumes file names have the format:
    "RegionName_lst16days_YYYY-MM-DD.tif"
    
    For example, for "Giao_Lac_lst16days_2022-12-20.tif", the region name is "Giao_Lac".
    """
    region_dict = {}
    # Loop over all files in the folder.
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".tif"):
            # Extract region name by splitting on "_lst16days"
            region_name = filename.split("_lst16days")[0].replace("_", "") + "_GiaoThuy_NamDinh"
            tif_file_path = os.path.join(folder_path, filename)
            try:
                coordinates = get_region_coordinates(tif_file_path)
                region_dict[region_name] = coordinates
            except Exception as e:
                print(f"Error processing file {filename}: {e}")
    return region_dict

# Example usage:
folder_path = "/mnt/data1tb/LSTRetrieval/Code/LST"  # Replace with your folder path
regions = read_region_coordinates(folder_path)
print(regions)


{'GiaoHuong_GiaoThuy_NamDinh': [[11860343.702493362, 2306440.893208176], [11860343.702493362, 2310602.437344628], [11855509.712762943, 2310602.437344628], [11855509.712762943, 2306440.893208176], [11860343.702493362, 2306440.893208176]], 'GiaoThanh_GiaoThuy_NamDinh': [[11858917.079295851, 2303624.0944356215], [11858917.079295851, 2308488.1703431844], [11854525.180106109, 2308488.1703431844], [11854525.180106109, 2303624.0944356215], [11858917.079295851, 2303624.0944356215]], 'BinhHoa_GiaoThuy_NamDinh': [[11853532.604562165, 2302129.7995674144], [11853532.604562165, 2307119.8200998907], [11849269.106611567, 2307119.8200998907], [11849269.106611567, 2302129.7995674144], [11853532.604562165, 2302129.7995674144]], 'HongThuan_GiaoThuy_NamDinh': [[11856050.394815147, 2303457.696154827], [11856050.394815147, 2309750.6347850305], [11850048.042250637, 2309750.6347850305], [11850048.042250637, 2303457.696154827], [11856050.394815147, 2303457.696154827]], 'BachLong_GiaoThuy_NamDinh': [[11848619.8

In [2]:
roi_regions_folder = "/mnt/data1tb/LSTRetrieval/Code/LST"
roi_regions = read_region_coordinates(roi_regions_folder)
for roi_region in roi_regions.keys():
    roi_name, roi = roi_region, roi_regions[roi_region]
    # print(roi_name.split("_")[0])


In [ ]:
from lst_retrieval import lst_retrive
from rvi_retrieval import main_rvi
from ndvi_retrieval import main_ndvi
import ee

ee.Initialize()

date_start = '2022-12-15'
date_end = '2025-01-01'
start_date = ee.Date(date_start)
end_date = ee.Date(date_end)
big_folder = "/mnt/data1tb/LSTRetrieval/Code/download_data"
roi_regions_folder = "/mnt/data1tb/LSTRetrieval/Code/LST"
roi_regions = read_region_coordinates(roi_regions_folder)

for roi_region in roi_regions.keys():
    roi_name, roi = roi_region, roi_regions[roi_region]
    geometry = ee.Geometry.Polygon(
    [roi],
    proj='EPSG:3857'
    )
    # print(roi_name, geometry)
    lst_retrive(date_start, date_end, geometry, roi_name, big_folder)
    main_rvi(start_date, end_date, geometry, big_folder, roi_name)
    main_ndvi(start_date, end_date, geometry, roi_name, big_folder)

Landsat Collection: {'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'SR_B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7561, 7721], 'crs': 'EPSG:32648', 'crs_transform': [30, 0, 566385, 0, -30, 2353215]}, {'id': 'SR_B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7561, 7721], 'crs': 'EPSG:32648', 'crs_transform': [30, 0, 566385, 0, -30, 2353215]}, {'id': 'SR_B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7561, 7721], 'crs': 'EPSG:32648', 'crs_transform': [30, 0, 566385, 0, -30, 2353215]}, {'id': 'SR_B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7561, 7721], 'crs': 'EPSG:32648', 'crs_transform': [30, 0, 566385, 0, -30, 2353215]}, {'id': 'SR_B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': 